# Nowcasting IMERG data

This notebook provide instructions on how to nowcast using the data that was previously downloaded. This notebook will cover how to 

1. Use the dataloader from the h5 files to produce windowed inputs
2. Initialize config files for each nowcasting model
3. Nowcast the generated inputs and save the outputs in a format ready to be used for downstream tasks

Author: Akshay Aravamudan (aaravamudan2014@my.fit.edu)

Last Edited: Jan 21, 2025

## Imports

In [1]:
from servir.utils.evaluation import generate_outputs
from servir.core.data_provider import IMERGDataModule

Pysteps configuration file found at: /volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/pysteps/pystepsrc



/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize parameters

In [2]:
# event id for which the data was downloaded
event_id = 1

# location of the h5 file that was generated after downloading the data
h5_dataset_location = '../data/events/'+str(event_id)+'.h5'

# as of now, we do not have IR data, so we set it None
ir_h5_dataset_location = None

# this string is used to determine the kind of dataloader we need to use
# for processing individual events, we reccommend the user to keep this fixed
dataset_type = 'wa'

## Initialize the dataloader

In [3]:
data_provider =  IMERGDataModule(
        forecast_steps = 12,
        history_steps = 12,
        imerg_filename = h5_dataset_location,
        ir_filename = ir_h5_dataset_location,
        batch_size = 32,
        image_shape = (360, 516),
        normalize_data=False,
        dataset = dataset_type)

# for now we are treating the test dataloader as the main one since we are only interested in predicting for individual events
data_loader = data_provider.test_dataloader()

original shape (49, 360, 516)
Precipitation Dataset input shape:  (26, 12, 1, 360, 516)
Precipitation Dataset output shape:  (26, 12, 1, 360, 516)


## Initialize config files for individual models and predict

For producing teh nowcasts using each of the models, the following parameters need to be initialized

1. model_type: name of the model ('convlstm', 'linda', 'steps', 'lagrangian', 'naive')
2. model_config_location: location where the python file with model specific parameters can be found. These have already been populated in the `configs/wa_imerg/` directory.
3. model_save_location: if the model requires a saved object (for instance neural network weights), this parameter points to that location.
4. use_gpu: flag for whether the model should use a GPU. note that this is only applicable for neural network models and the node which runs this notebook must contain a GPU.


### ConvLSTM

In [4]:
model_type = 'convlstm'
model_config_location = 'configs/wa_imerg/ConvLSTM.py'
model_save_location = 'temp/imerg_only_mse_params.pth'
model_output_location = 'results/'
use_gpu = False
produce_ensemble_outputs = False
convlstm_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id, model_output_location=model_output_location)


loading config from configs/wa_imerg/ConvLSTM.py ...


/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0
0


/vol_efthymios/NFS07/en279/SERVIR/TITO_test3/ML/nowcasting/servir/core/model_picker.py:115: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(Y, dtype=torch.float32, device= self.config['device'])


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


### STEPS

In [5]:
model_type = 'steps'
model_config_location = 'configs/wa_imerg/PySTEPS.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
steps_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)


loading config from configs/wa_imerg/PySTEPS.py ...


/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0
Computing STEPS nowcast
-----------------------

Inputs
------
input dimensions: 360x516
km/pixel:         10
time step:        30 minutes

Methods
-------
extrapolation:          semilagrangian
bandpass filter:        gaussian
decomposition:          fft
noise generator:        nonparametric
noise adjustment:       no
velocity perturbator:   bps
conditional statistics: no
precip. mask method:    incremental
probability matching:   cdf
FFT method:             numpy
domain:                 spatial

Parameters
----------
number of time steps:     12
ensemble size:            1
parallel threads:         1
number of cascade levels: 6
order of the AR(p) model: 2
velocity perturbations, parallel:      10.88,0.23,-7.68
velocity perturbations, perpendicular: 5.76,0.31,-2.72
precip. intensity threshold: -10.0
************************************************
* Correlation coefficients for cascade levels: *
************************************************
--------

### Lagrangian Persistence

In [6]:

model_type = 'lagrangian'
model_config_location = 'configs/wa_imerg/lagrangian_persistence.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
lagrangian_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)


loading config from configs/wa_imerg/lagrangian_persistence.py ...


/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0


### Naive Persistence

In [7]:
model_type = 'naive'
model_config_location = 'configs/wa_imerg/naive_persistence.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
naive_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)

loading config from configs/wa_imerg/naive_persistence.py ...


/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.008127450942993164  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.002798318862915039  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.0024907588958740234  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.0021741390228271484  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.002173900604248047  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.002203226089477539  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields took  0.002493619918823242  seconds
The shape of the resulting array is:  (12, 360, 516)
Advecting the radar rainfall fields t

### LINDA

In [ ]:
model_type = 'linda'
model_config_location = 'configs/wa_imerg/LINDA.py'
model_save_location = ''
use_gpu = False
produce_ensemble_outputs = False
model_output_location = 'results/'
naive_output = generate_outputs(data_loader, model_type, model_config_location, model_save_location, use_gpu, produce_ensemble_outputs=produce_ensemble_outputs, event_name = event_id,model_output_location = model_output_location)


loading config from configs/wa_imerg/LINDA.py ...


/volume/NFS/aa3328/miniconda3/envs/tito_env/lib/python3.12/site-packages/torch/utils/data/_utils/collate.py:285: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


starting predictions for batch 0
Computing LINDA nowcast
-----------------------

Inputs
------
dimensions:           360x516
number of time steps: 12

Methods
-------
nowcast type:         deterministic
feature detector:     blob
extrapolator:         semilagrangian
kernel type:          anisotropic

Parameters
----------
number of time steps:       12
ARI model order:            1
localization window radius: 72.0

Detecting features... found 15 blobs.
Transforming to Lagrangian coordinates... done.
Estimating the first convolution kernel... done.
Estimating the ARI(p,1) parameters... done.
Estimating the second convolution kernel... done.
Computing nowcast for time step 1... done.
Computing nowcast for time step 2... done.
Computing nowcast for time step 3... done.
Computing nowcast for time step 4... done.
Computing nowcast for time step 5... done.
Computing nowcast for time step 6... done.
Computing nowcast for time step 7... done.
Computing nowcast for time step 8... done.
Computi